In [3]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
import plotly.graph_objects as go

In [21]:
# environment

load_dotenv(override=True)
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [23]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [7]:
# Already trained in Smart Pricer project:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [9]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

In [13]:
# Lets Create a chroma Datastore

client = chromadb.PersistentClient(path=DB)

In [15]:
# Check if the collection exists and delete it if it does
collection_name = "products"

# For old versions of Chroma, use this line instead of the subsequent one
# existing_collection_names = [collection.name for collection in client.list_collections()]
existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

In [27]:
# The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to a 384 dimensional dense vector space and is ideal for tasks like semantic search.

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]
vector

array([-9.46716070e-02,  4.27619554e-02,  5.51620610e-02, -5.11038932e-04,
        1.16203483e-02, -6.80130422e-02,  2.76405923e-02,  6.06974959e-02,
        2.88530793e-02, -1.74127668e-02, -4.94346246e-02,  2.30993181e-02,
       -1.28614483e-02, -4.31402661e-02,  2.17510145e-02,  4.26548496e-02,
        5.10499962e-02, -7.79727325e-02, -1.23247221e-01,  3.67455408e-02,
        4.54109861e-03,  9.47937891e-02, -5.53099066e-02,  1.70641504e-02,
       -2.92872898e-02, -4.47124504e-02,  2.06784457e-02,  6.39320314e-02,
        2.27427669e-02,  4.87789884e-02, -2.33496190e-03,  4.72859368e-02,
       -2.86259241e-02,  2.30624415e-02,  2.45130546e-02,  3.95681746e-02,
       -4.33176309e-02, -1.02316640e-01,  2.79877288e-03,  2.39304788e-02,
        1.61556229e-02, -8.99080094e-03,  2.07255688e-02,  6.40122816e-02,
        6.89179301e-02, -6.98360875e-02,  2.89759203e-03, -8.10988992e-02,
        1.71122849e-02,  2.50654924e-03, -1.06529042e-01, -4.87733148e-02,
       -1.67762004e-02, -

In [31]:
# Quick sidebar - a function to compare vectors

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [33]:
# And let's see how adding a few words to the context can change things up!

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

Similarity between Java and C++ is 50.7%
Similarity between Java and mug is 25.8%
Similarity between Cup of Java and mug is 49.3%


In [35]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [37]:
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

In [39]:
# The next cell populates the 400,000 items in Chroma.

NUMBER_OF_DOCUMENTS = len(train)

# Uncomment if you'd rather not wait for the full 400,000
# NUMBER_OF_DOCUMENTS = 20000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [19:26<00:00,  2.92s/it]
